<a href="https://colab.research.google.com/github/WelberFerreira/sistema-de-gerenciamento/blob/master/Trend_Test_CHIRPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Source
http://support.sas.com/kb/33/092.html
Mendenhall, Scheaffer, and Wackerly (1986), Mathematical Statistics with Applications, 3rd Ed., Duxbury Press, CA.

Wald, A. and Wolfowitz, J. (1940), "On a test whether two samples are from the same population," Ann. Math Statist. 11, 147-162

# **1. Importing Libraries**

In [30]:
import wald_wolowitz_cox
import mannKendal
import math
import pandas as pd
from scipy.stats import binom
from scipy import stats
import sys
import os

sys.path.insert(0,'/content/drive/MyDrive/CHIRPS/testes')

# **2. Cox Test Function**

In [31]:
def Cox_test(sef_of_values, alpha):
    # Source: http://statistic-on-air.blogspot.com/2009/08/trend-analysis-with-cox-stuart-test-in.html
    n = int(len(sef_of_values))

    #cust_even
    if n%2!=0: #deleting central element
        middle = math.trunc(n/2)
        del sef_of_values[middle]
        n=n-1

    # Divide in 2 sets
    fHalf = sef_of_values[:(int(n/2))]
    sHalf = sef_of_values[(int(n/2)):]

    difference = [a_i - b_i for a_i, b_i in zip(fHalf, sHalf)]

    neg = len(list(filter(lambda x: (x < 0), difference)))
    zero = len(list(filter(lambda x: (x == 0), difference)))
    pos  = len(list(filter(lambda x: (x > 0), difference)))

    p = neg+pos

    if neg < pos:
        smaller_signal = neg
    else:
        smaller_signal = pos

    #u = norm.ppf(1-0.05/2)

    cdf = binom.cdf(smaller_signal, p, alpha*10)

    Hypo = " "
    alpha = alpha
    if cdf < alpha:
        if neg > pos:
            Hypo = "increasing"
        else:
            Hypo = "decreasing"
    else:
        Hypo = "no trend"


    return(cdf, Hypo, pos, neg)


# **3. Obtaining Paths**

In [32]:
# folder path
dir_path = r'/content/drive/MyDrive/CHIRPS/Results'
result_path = r'/content/drive/MyDrive/CHIRPS'
output_folder = os.path.join(result_path, "Trend_Results")

# list to store files
arquivos = []

# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        arquivos.append(path)

# **4. Applying Trend Tests**

In [ ]:
for arquivo in arquivos:

        input_file = os.path.join(dir_path, arquivo)
        df = pd.read_csv(input_file, sep=';')

        result = "result_" + arquivo
        output_file = os.path.join(output_folder, result)

        result = open(output_file, "w+")

        #retriving the header
        #header = list(df.columns.values)
        header = df.columns.values.tolist()
        print (header)
        header.pop(0)


        #Loop into the columns
        print ("Test de Mann-Kendall, Wald-Wolfowitz, Cox-Stuart and Spearman")
        alfa_ = 0.05

        result.write("Station,n,Trend_MK, boolean_MK,z_MK,p_MK,z_WW,p_1_WW, \
        Trend_1_WW, p_2_WW, Trend_2_WW, Direction_WW,p_cox,Trend,\
                     Positive_values_cox,Negative_values_cox,rho,p_rho, Trend")
        result.write("\n")

        for col in header:

                # Mann-Kendall Test
                trend = df[col] #retrieving the column
                test_trend,h,p,z = mannKendal.mk_test(trend,alpha=alfa_)
                a = "{},{},{},{},{:.5f},{:.5f}".format(col,trend.shape[0],
                                                       test_trend, h,z,p)
                result.write(a)

                #Wald-Wolfowitz test
                z,p_1Tail,trend_1,p_2Tail,trend_2,direction = wald_wolowitz_cox.WW_test(trend,alfa_)
                b = ",{:.5f},{:.5f},{},{:.5f},{},{}".format(z,p_1Tail, trend_1,p_2Tail,trend_2,direction)
                result.write(b)

                #Cox-Stuart test
                p_value, trend_information, pos, neg = Cox_test(trend,alfa_)
                c = ",{:.5f},{},{},{}".format(p_value, trend_information,pos,neg)
                result.write(c)

                #Spearman Test
                n = len(trend)
                idx = [i+1 for i in range(n)]
                rho,p_rho = stats.spearmanr(trend,idx,axis=0)
                if p_rho < alfa_:
                        trend_rho = "Trending"
                else:
                        trend_rho = "No Trend"
                d = ",{:.5f},{:.5f},{}\n".format(rho,p_rho, trend_rho)
                result.write(d)

        result.close()